Student Name: Douglas Steen

Instructor: James Irving, PhD

Date of Submittal:

Date of Project Review:

Cohort: ds-ft-10072019

# Project Overview 
The purpose of this study is to  perform specific hypothesis tests to gain actionable insights that can be used by Northwind managment to improve business strategy and efficiency. The study aims to answer the following questions, and provide recommendations based on the results.

Question # 1:
Does discount amount have a statistically significant effect on the quantity of a product in an order? If so, at what level(s) of discount?

Question # 2:
Does the presence of a picture result in a statistically signficant increase in the rate of product orders? (units/month?)

Question # 3:
Does the shipper used have a statistically significant effect on the timeliness of shipping? (Shipped date +- required date)

Question # 4:
Does freight cost have a stastically signficant effect on the number of orders a specific customer makes?

# Import Libraries & Connect to Database

In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
import scipy.stats as stats
import numpy as np

In [2]:
# Instantiate a cursor object and connect to the Northwind sqlite database

conn = sqlite3.connect('Northwind_small.sqlite')
cur = conn.cursor()

# Question # 1: Discount & Order Quantity

Question: Does discount amount have a statistically significant effect on the quantity of a product in an order? If so, at what level(s) of discount?

Null Hypothesis: Discount amount has no effect on the quantity of a product in an order.

Alt. Hypothesis: When the discount amount is higher, the quantity of a product in an order is higher than if there is no/low discount.

## Obtain

Obtain necessary data from Northwind SQL database to answer Question #1

-Want to visualize all variables in the OrderDetail table


In [36]:
# Obtain OrderDetail table, create a pandas dataframe for all entries with no discount
cur.execute("""SELECT * FROM OrderDetail WHERE Discount == 0;""")
q1_df_nd = pd.DataFrame(cur.fetchall())
q1_df_nd.columns = [x[0] for x in cur.description]
q1_df_nd.drop(labels='Id', inplace=True, axis=1)
q1_df_nd.describe()

,OrderId,ProductId,UnitPrice,Quantity,Discount
count,1317.000000,1317.000000,1317.000000,1317.000000,1317.0
mean,10662.791951,41.396355,26.171534,21.715262,0.0
std,244.835366,22.217410,29.717354,17.507493,0.0
min,10248.000000,1.000000,2.000000,1.000000,0.0
25%,10448.000000,23.000000,12.000000,10.000000,0.0
50%,10671.000000,41.000000,18.400000,18.000000,0.0
75%,10879.000000,60.000000,32.800000,30.000000,0.0
max,11077.000000,77.000000,263.500000,130.000000,0.0


In [37]:
# Obtain OrderDetail table, create a pandas dataframe with discount 5% or greater
cur.execute("""SELECT * FROM OrderDetail WHERE Discount >= 0.05;""")
q1_df2 = pd.DataFrame(cur.fetchall())
q1_df2.columns = [x[0] for x in cur.description]
q1_df2.drop(labels='Id', inplace=True, axis=1)
q1_df2.describe()

,OrderId,ProductId,UnitPrice,Quantity,Discount
count,831.000000,831.000000,831.000000,831.000000,831.000000
mean,10650.444043,39.892900,26.264801,27.323706,0.145439
std,233.645698,22.016846,30.074243,20.726850,0.071269
min,10250.000000,1.000000,2.000000,1.000000,0.050000
25%,10451.000000,21.000000,12.250000,12.000000,0.100000
50%,10643.000000,40.000000,18.000000,20.000000,0.150000
75%,10844.000000,59.000000,31.230000,36.000000,0.200000
max,11077.000000,77.000000,263.500000,130.000000,0.250000
